In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import torch
import torch_geometric
from torch_geometric.nn import MessagePassing
import  random
import math

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(0)

In [ ]:
#######ciao10
data_name = 'ciao'
core = 10
completion_type = 'full_mf'
device = 'cuda'
alpha1 = 0.1
alpha2 = 0.2
alpha3 = 1.0 - alpha1 - alpha2
completion_dim = 20
indi_scale = 0.25
neigh_scale = 0.25
att_scale = 1.0
beta = 20.0
power = 0.5
reced_data = torch.from_numpy(np.loadtxt(f'../../datasets/{data_name}/{data_name}{core}{completion_dim}{alpha1}{alpha2}{alpha3}_{completion_type}_exposed_data.txt')).long().to(device)
# reced_data = torch.from_numpy(np.loadtxt(f'../../../interference/datasets/ciao/save3/ciao10_full_mf_exposed_data.txt')).long().to(device)

In [ ]:
# data_name = 'filmtrust'
# core = 5
# completion_type = 'full_mf'
# device = 'cuda'
# alpha1 = 0.2
# alpha2 = 0.65
# alpha3 = 0.15
# completion_dim = 20
# indi_scale = 0.25
# neigh_scale = 0.25
# att_scale = 1.0
# beta = 20.0
# power = 0.5
# reced_data = torch.from_numpy(np.loadtxt(f'../../datasets/{data_name}/{data_name}{core}{completion_dim}{alpha1}{alpha2}{alpha3}_{completion_type}_exposed_data.txt')).long().to(device)
# # reced_data = torch.from_numpy(np.loadtxt(f'../../../interference/datasets/ciao/save3/ciao10_full_mf_exposed_data.txt')).long().to(device)

In [ ]:
# data_name = 'epinions'
# core = 10
# completion_type = 'full_mf'
# device = 'cuda'
# alpha1 = 0.2
# alpha2 = 0.6
# alpha3 = 0.2
# completion_dim = 20
# indi_scale = 0.25
# neigh_scale = 0.25
# att_scale = 1.0
# beta = 30.0
# power = 0.5
# reced_data = torch.from_numpy(np.loadtxt(f'../../datasets/{data_name}/{data_name}{core}{completion_dim}{alpha1}{alpha2}{alpha3}_{completion_type}_exposed_data.txt')).long().to(device)
# # reced_data = torch.from_numpy(np.loadtxt(f'../../../interference/datasets/ciao/save3/ciao10_full_mf_exposed_data.txt')).long().to(device)

In [ ]:
print('reced_data', reced_data, reced_data.shape)

In [ ]:
class click_conv(MessagePassing):
    def __init__(self):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5).

        # [num_edge, 2]
        trust_network = np.loadtxt(f'../../datasets/{data_name}/processed_trust{core}.txt').astype(int)
        print('trust_network', trust_network, trust_network.shape)
        # [2, num_edge] the first row contains source nodes, the second row contains target nodes
        self.edge_index = torch.from_numpy(trust_network[:, [1, 0]]).long().t().contiguous().to(device)

        user_emb_all = torch.from_numpy(np.loadtxt(f'../para/{data_name}{core}{completion_dim}_{completion_type}_user_emb.txt')).float().to(device)
        item_emb_all = torch.from_numpy(np.loadtxt(f'../para/{data_name}{core}{completion_dim}_{completion_type}_item_emb.txt')).float().to(device)
        
        # user_emb_all = torch.from_numpy(np.loadtxt(f'../../../interference/simulation/para/save3/ciao10_full_mf_user_emb.txt')).float().to(device)
        # item_emb_all = torch.from_numpy(np.loadtxt(f'../../../interference/simulation/para/save3/ciao10_full_mf_item_emb.txt')).float().to(device)
        
        print('user_emb_all', user_emb_all, user_emb_all.shape, torch.norm(user_emb_all, dim=1), torch.norm(user_emb_all, dim=1).max(), torch.norm(user_emb_all, dim=1).min())
        print('item_emb_all', item_emb_all, item_emb_all.shape, torch.norm(item_emb_all, dim=1), torch.norm(item_emb_all, dim=1).max(), torch.norm(item_emb_all, dim=1).min())

        
        self.user_esmb_all = user_emb_all

        self.num_users = len(user_emb_all)
        self.num_items = len(item_emb_all)

        print('num_users', self.num_users)
        print('num_items', self.num_items)

        # [num_users, 1]
        self.full_rating = torch.matmul(user_emb_all, item_emb_all.t())
        print('full_rating', self.full_rating)
        # [num_edge, 1]
        self.sim_per_edge = torch.sigmoid((user_emb_all[self.edge_index[0]] * user_emb_all[self.edge_index[1]]).sum(dim=1, keepdim=True) * att_scale)
        self.in_degree = torch_geometric.utils.degree(self.edge_index[1], self.num_users).float()
        self.in_norm = torch.pow(self.in_degree, power)
        self.in_norm[self.in_norm == 0.0] = 1.0

        print('in_degree', self.in_degree, self.in_degree.shape)
        print('in_norm', self.in_norm, self.in_norm.shape)


    def click_forward(self, oi, target_item):
        # [num_users, sample_size]
        indi_prefer = ((self.full_rating[:, target_item] - 3.0) * indi_scale).reshape(-1, 1)
        neigh_prefer = ((self.full_rating[:, target_item] - 3.0) * neigh_scale).reshape(-1, 1)

        prefer_factor = oi * neigh_prefer

        # [num_users, sample_size]
        neigh_effect = self.propagate(edge_index=self.edge_index, x=prefer_factor, coeff=self.sim_per_edge)
        # [num_users, sample_size]
        temp_click_prob = indi_prefer + beta * neigh_effect
        click_prob = torch.sigmoid(temp_click_prob)


        return click_prob.reshape(1, -1)[0], temp_click_prob.reshape(1, -1)[0], indi_prefer.reshape(1, -1)[0], (beta * neigh_effect).reshape(1, -1)[0]


    def message(self, x_j, coeff):
        return x_j * coeff / self.in_norm[self.edge_index[1]].reshape(-1, 1)




In [ ]:
click_model = click_conv()
click_prob = torch.zeros(click_model.num_users, click_model.num_items, device=device)
non_restrict_prob = torch.zeros(click_model.num_users, click_model.num_items, device=device)
indi_prefer_cache = torch.zeros(click_model.num_users, click_model.num_items, device=device)
neigh_prefer_cache = torch.zeros(click_model.num_users, click_model.num_items, device=device)
for i in range(click_model.num_items):
    oi = torch.zeros(click_model.num_users, 1, device=device)
    i_reced = reced_data[reced_data[:, 1]==i]
    oi[i_reced[:, 0]] = 1.0
    click_prob[:, i], non_restrict_prob[:, i], indi_prefer_cache[:, i], neigh_prefer_cache[:, i] = click_model.click_forward(oi, i)
print('click_prob', click_prob)
print('non_restrict_prob', non_restrict_prob)
print('indi_prefer_cache', indi_prefer_cache)
print('neigh_prefer_cache', neigh_prefer_cache)

In [ ]:
reced_click_prob = click_prob[reced_data[:, 0], reced_data[:, 1]]
assert torch.any(reced_click_prob <= 1.0) or torch.any(reced_click_prob >= 0.0)
reced_non_restrict_prob = non_restrict_prob[reced_data[:, 0], reced_data[:, 1]]
reced_indi_prefer = indi_prefer_cache[reced_data[:, 0], reced_data[:, 1]]
reced_neigh_prefer = neigh_prefer_cache[reced_data[:, 0], reced_data[:, 1]]
label = (reced_click_prob>0.5).float()
print('reced_click_prob', reced_click_prob, reced_click_prob.min(), reced_click_prob.max())
print('reced_non_restrict_prob', reced_non_restrict_prob, reced_non_restrict_prob.min(), reced_non_restrict_prob.max())
print('reced_indi_prefer', reced_indi_prefer, reced_indi_prefer.min(), reced_indi_prefer.max())
print('reced_neigh_prefer', reced_neigh_prefer, reced_neigh_prefer.min(), reced_neigh_prefer.max())

In [ ]:
pos = label.sum()
neg = (len(reced_data) - label.sum())
print(pos, neg)

In [ ]:
reced_data = reced_data.cpu().numpy()
label = label.cpu().numpy()
reced_click_prob = reced_click_prob.cpu().numpy()
reced_click_prob = np.hstack((reced_data, reced_click_prob.reshape(-1, 1)))

reced_non_restrict_prob = reced_non_restrict_prob.cpu().numpy()
reced_non_restrict_prob = np.hstack((reced_data, reced_non_restrict_prob.reshape(-1, 1)))

labeled_gen_data = np.hstack((reced_data, label.reshape(-1, 1)))
print('labeled_gen_data', labeled_gen_data, labeled_gen_data.shape)
print('reced_click_prob', reced_click_prob, reced_click_prob.shape)
print('reced_non_restrict_prob', reced_non_restrict_prob, reced_non_restrict_prob.shape)

In [ ]:
if beta == 0.0:
    np.savetxt(f'../../datasets/{data_name}/{data_name}{core}{alpha1}{alpha2}{alpha3}{indi_scale}{neigh_scale}{att_scale}{beta}{power}_{completion_type}_indi_sim_data.txt', labeled_gen_data, fmt='%d')
    np.savetxt(f'../../datasets/{data_name}/{data_name}{core}{alpha1}{alpha2}{alpha3}{indi_scale}{neigh_scale}{att_scale}{beta}{power}_{completion_type}_indi_sim_data_wrt_click_prob.txt', reced_click_prob)
    np.savetxt(f'../../datasets/{data_name}/{data_name}{core}{alpha1}{alpha2}{alpha3}{indi_scale}{neigh_scale}{att_scale}{beta}{power}_{completion_type}_indi_sim_data_wrt_non_restrict_click_prob.txt', reced_non_restrict_prob)
else:
    np.savetxt(f'../../datasets/{data_name}/{data_name}{core}{alpha1}{alpha2}{alpha3}{indi_scale}{neigh_scale}{att_scale}{beta}{power}_{completion_type}_sim_data.txt', labeled_gen_data, fmt='%d')
    np.savetxt(f'../../datasets/{data_name}/{data_name}{core}{alpha1}{alpha2}{alpha3}{indi_scale}{neigh_scale}{att_scale}{beta}{power}_{completion_type}_sim_data_wrt_click_prob.txt', reced_click_prob)
    np.savetxt(f'../../datasets/{data_name}/{data_name}{core}{alpha1}{alpha2}{alpha3}{indi_scale}{neigh_scale}{att_scale}{beta}{power}_{completion_type}_sim_data_wrt_non_restrict_click_prob.txt', reced_non_restrict_prob)


In [ ]:
plt.hist(click_model.sim_per_edge.cpu().numpy(), bins=20)

In [ ]:
click_model.sim_per_edge.min()